# Document Question Answering

An example of using Chroma DB and LangChain to do question answering over documents.

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader

## Load documents

Load documents to do question answering over. If you want to do this over your documents, this is the section you should replace.

In [2]:
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

## Split documents

Split documents into small chunks. This is so we can find the most relevant chunks for a query and pass only those into the LLM.

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Initialize ChromaDB

Create embeddings for each chunk and insert into the Chroma vector database.

In [6]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get('OPENAI_API_KEY'))
#vectordb = Chroma.from_documents(texts, embeddings)

## Create the chain

Initialize the chain we will use for question answering.

In [14]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=os.environ.get('OPENAI_API_KEY')), chain_type="stuff", vectorstore=vectordb)

/home/ubuntu/miniconda3/envs/lc/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:252: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


## Ask questions!

Now we can use the chain to ask questions!

In [15]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The President said that Ketanji Brown Jackson is one of our nation's top legal minds and will continue Justice Breyer's legacy of excellence. She is a former top litigator in private practice, former federal public defender, and from a family of public school educators and police officers. Since she's been nominated, she's received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

# Forms of Conversational Memory

## ConversationBufferMemory

`The prompt attempts to reduce hallucinations (where a model makes things up) by stating:
"If the AI does not know the answer to a question, it truthfully says it does not know."`

In [23]:
from langchain import OpenAI
from langchain.chains import ConversationChain
import os
# first initialize the large language model
llm = OpenAI(
	openai_api_key=os.environ.get('OPENAI_API_KEY'),
)

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)

/home/ubuntu/miniconda3/envs/lc/lib/python3.10/site-packages/langchain/utils/utils.py:149: UserWarning: WARNING! deployment is not default parameter.
                deployment was transferred to model_kwargs.
                Please confirm that deployment is what you intended.
  warnings.warn(
/home/ubuntu/miniconda3/envs/lc/lib/python3.10/site-packages/langchain/utils/utils.py:149: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(


In [24]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [31]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm)
)

In [29]:
print(conversation.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [32]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum, 
    "Good morning AI!"
)

InvalidRequestError: Invalid URL (POST /v1/openai/deployments/text-davinci-003/completions)

In [18]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [19]:
count_tokens(
    conversation_buf, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

InvalidRequestError: Must provide an 'engine' or 'deployment_id' parameter to create a <class 'openai.api_resources.completion.Completion'>